# pandas Quiz-02

## 구분자로 구분된 문자열 컬럼 

(2021.8.17)

- 몇가지 배울 내용

 - explode()
 - map()
 - lamda 함수

**(Question-1)** 아래 author 컬럼  저자정보에서, Org1에서 나온 문서 개수를 구하시오. 눈으로 대충 세워서.

> (1) 2개 (2) 4개

In [117]:
import pandas as pd
from pandas import DataFrame, Series

df = DataFrame( {'id':[1,2,3], 'author_info':['Kim_Org1;Lee_Org1;Noh_Org1', 'Kim_Org1;Jang_Org2', 'Lee_Org3']})
df

,id,author_info
0,1,Kim_Org1;Lee_Org1;Noh_Org1
1,2,Kim_Org1;Jang_Org2
2,3,Lee_Org3


### map()

In [132]:
df['author_info_list'] = df['author_info'].map(lambda x : x.split(';'))
df

,id,author_info,author_info_list
0,1,Kim_Org1;Lee_Org1;Noh_Org1,"[Kim_Org1, Lee_Org1, Noh_Org1]"
1,2,Kim_Org1;Jang_Org2,"[Kim_Org1, Jang_Org2]"
2,3,Lee_Org3,[Lee_Org3]


### explode()

In [133]:
df2=df.explode('author_info_list')
df2

,id,author_info,author_info_list
0,1,Kim_Org1;Lee_Org1;Noh_Org1,Kim_Org1
0,1,Kim_Org1;Lee_Org1;Noh_Org1,Lee_Org1
0,1,Kim_Org1;Lee_Org1;Noh_Org1,Noh_Org1
1,2,Kim_Org1;Jang_Org2,Kim_Org1
1,2,Kim_Org1;Jang_Org2,Jang_Org2
2,3,Lee_Org3,Lee_Org3


### str.split()

In [134]:
df2[["author","org"]] = df2["author_info_list"].str.split(pat="_", expand=True)
df2

,id,author_info,author_info_list,author,org
0,1,Kim_Org1;Lee_Org1;Noh_Org1,Kim_Org1,Kim,Org1
0,1,Kim_Org1;Lee_Org1;Noh_Org1,Lee_Org1,Lee,Org1
0,1,Kim_Org1;Lee_Org1;Noh_Org1,Noh_Org1,Noh,Org1
1,2,Kim_Org1;Jang_Org2,Kim_Org1,Kim,Org1
1,2,Kim_Org1;Jang_Org2,Jang_Org2,Jang,Org2
2,3,Lee_Org3,Lee_Org3,Lee,Org3


### org 카운팅

#### 총등장횟수

In [135]:
df2['org'].value_counts()

Org1    4
Org2    1
Org3    1
Name: org, dtype: int64

#### 유니크 등장횟수

문서에 한번 이상 출현해도 1번 출현한 것으로 간주함

In [136]:
def fun1(x):
    #print(x)
    return  sorted(set( ';'.join(x).split(sep)))

s3=df2.groupby(['id'])['org'].apply(lambda x : fun1(x))

df3 = DataFrame(s3)
df3

,org
id,
1,[Org1]
2,"[Org1, Org2]"
3,[Org3]


In [137]:
df4 = df3.explode('org')
df4

,org
id,
1,Org1
2,Org1
2,Org2
3,Org3


In [138]:
df4.value_counts()

org 
Org1    2
Org2    1
Org3    1
dtype: int64

즉 Org1에서 2개 문서가 생산했다. 

### 함수 작성

In [139]:
import numpy as np

def value_counts(df, col_index='id', colname='country', *, unique=True, sep=';') -> Series:
    """
    2021.8.17
    
    parameters
    ----------
    df : input DataFrame
    col_index : 구분자 없는 컬럼
    colname : 구분자 있는 컬럼 
    
    returns
    -------
    Series
    """
    #df[colname]=df[colname].astype(str)
    
    def fun1(x, *, sep=';', unique=True):
        x2=[]
        for each in x:
            if pd.isna(each) == True:
                pass
            elif f"{each}".strip() == '':
                pass
            else:
                x2.append(f"{each}")
        
        if unique == True:
            return sorted(set( f'{sep}'.join( x2).split(f'{sep}')))
        else:
            return  f'{sep}'.join(x).split(f'{sep}')

    
    s1=df.groupby([col_index])[colname].apply(lambda x :  fun1(x, sep=sep, unique=unique))
           
    df_s1 = DataFrame(s1)
    
    df_s2 = df_s1.explode(colname)
    
    df_s3 = df_s2.dropna(subset=[colname])
    s3     = df_s3[colname].value_counts()
    
    return s3

In [140]:
value_counts(df2, 'id','org', unique=True)

Org1    2
Org2    1
Org3    1
Name: org, dtype: int64

In [141]:
value_counts(df2, 'id','org', unique=False)

Org1    4
Org2    1
Org3    1
Name: org, dtype: int64